## Import necessary modules and data

In [2]:
import numpy as np
import pandas as pd
import sklearn
import keras
import seaborn
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
#!pip install -q keras_metrics
import keras_metrics
from sklearn.model_selection import cross_validate
#!pip install livelossplot
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import precision_recall_fscore_support as score
import random


Using TensorFlow backend.


## Wczytujemy poprzednio utworzone zbiory

In [3]:
X = pd.read_csv('../data/cleaned_data.csv', index_col=0)
y = pd.read_csv('../data/labels.csv', header = None)

X_test = pd.read_csv('../data/X_test.csv', index_col=0)
X_train = pd.read_csv('../data/X_train.csv', index_col=0)
X_train_std = pd.read_csv('../data/X_train_std.csv', index_col=0)
X_test_std = pd.read_csv('../data/X_test_std.csv', index_col=0)
y_train = pd.read_csv('../data/y_train.csv', index_col=0)
y_test = pd.read_csv('../data/y_test.csv', index_col=0)

Zbuduję przykładową sieć, która składa się z warstwy wejściowej, jednej warstwy ukrytej i  warstwy wyjściowej.
Kompilujemy model przy użyciu 

loss_function='binary_crossentropy';

optimizer='Adam'.


Naszą główną metryką będzie **precyzja** oznaczająca procent osób prawidłowie określonych jako zagrożone odejściem.

## Pierwsze podejście do modelu

In [4]:
model = Sequential()

model.add(Dense(11, activation='relu', input_shape=(10,)))
 
model.add(Dense(121, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(),'accuracy'])
                   
model.fit(X_train_std, y_train,epochs=3, batch_size=1, verbose=1, validation_data=(X_test,y_test))

model.evaluate(X_test_std,y_test)

Train on 6700 samples, validate on 3300 samples
Epoch 1/3
6700/6700 [==============================] - 20s 3ms/step - loss: 0.4150 - precision: 0.6900 - acc: 0.8303 - val_loss: 3.4115 - val_precision: 0.3043 - val_acc: 0.7882
Epoch 2/3
6700/6700 [==============================] - 20s 3ms/step - loss: 0.3655 - precision: 0.7421 - acc: 0.8512 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 3/3
3300/3300 [==============================] - 0s 30us/step


[0.3837403065508062, 0.6790123455113549, 0.8403030303030303]

## Sprawdźmy skuteczność naszej predykcji raportem klasyfikacyjnym


In [5]:
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test,y_pred)
report1 = classification_report(y_test,y_pred,output_dict=True)
print(report)
print(report1['0']['precision'])


              precision    recall  f1-score   support

           0       0.86      0.95      0.90      2628
           1       0.68      0.41      0.51       672

   micro avg       0.84      0.84      0.84      3300
   macro avg       0.77      0.68      0.71      3300
weighted avg       0.83      0.84      0.82      3300

0.86286701208981


### Sprawdźmy, jak prezentują się wyniki w zależności od liczby neuronów w warstwie ukrytej.

In [6]:
nodes = [10, 30, 50, 70, 90, 110]
wyniki = {}
for i in nodes:
    model = 0

    model = Sequential()

    model.add(Dense(11, activation='relu', input_shape=(10,)))

    model.add(Dense(i, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[keras_metrics.precision(), 'accuracy'])

    model.fit(X_train_std, y_train, epochs=5, batch_size=1, verbose=1, validation_data=(X_test, y_test))
    print("Zakończono trenowanie modelu z {0} węzłami.".format(i))
    model.evaluate(X_test_std, y_test)
    y_pred = model.predict_classes(X_test_std)
    report = classification_report(y_test, y_pred, output_dict=True)
    wyniki.update({i: [report['0']['precision'], report['1']['precision']]})

print(wyniki)


Train on 6700 samples, validate on 3300 samples
Epoch 1/5
6700/6700 [==============================] - 20s 3ms/step - loss: 0.4485 - precision: 0.5988 - acc: 0.8063 - val_loss: 6.9746 - val_precision: 0.1477 - val_acc: 0.5621
Epoch 2/5
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3752 - precision: 0.7279 - acc: 0.8455 - val_loss: 7.3863 - val_precision: 0.1465 - val_acc: 0.5376
Epoch 3/5
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3600 - precision: 0.7454 - acc: 0.8521 - val_loss: 7.5963 - val_precision: 0.1446 - val_acc: 0.5252
Epoch 4/5
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3564 - precision: 0.7663 - acc: 0.8561 - val_loss: 10.2539 - val_precision: 0.1924 - val_acc: 0.3576
Epoch 5/5
6700/6700 [==============================] - 20s 3ms/step - loss: 0.3526 - precision: 0.7623 - acc: 0.8569 - val_loss: 10.9511 - val_precision: 0.1942 - val_acc: 0.3136
Zakończono trenowanie modelu z 10 węzłami.
3300/3300 [======

## Wybieramy model z 90 węzłami - dał najlepsze wyniki

In [7]:
from keras.callbacks import EarlyStopping

model = Sequential()

model.add(Dense(10, activation='relu', input_shape=(10,)))
model.add(Dropout(0.1, ))

model.add(Dense(90, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(), 'accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train_std, y_train, epochs=30, batch_size=1, verbose=1, validation_data=(X_test, y_test),
          callbacks=[early_stopping_monitor])

model.evaluate(X_test_std, y_test)
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test, y_pred)
report1 = classification_report(y_test, y_pred, output_dict=True)
print(report)



Train on 6700 samples, validate on 3300 samples
Epoch 1/30
6700/6700 [==============================] - 22s 3ms/step - loss: 0.4562 - precision: 0.5911 - acc: 0.8024 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 2/30
6700/6700 [==============================] - 23s 3ms/step - loss: 0.3984 - precision: 0.6775 - acc: 0.8337 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 3/30
3300/3300 [==============================] - 0s 38us/step
              precision    recall  f1-score   support

           0       0.85      0.98      0.91      2628
           1       0.81      0.34      0.48       672

   micro avg       0.85      0.85      0.85      3300
   macro avg       0.83      0.66      0.69      3300
weighted avg       0.84      0.85      0.82      3300



## Zapisujemy wytrenowany model

In [9]:
model.save_weights("../saved_models/model_1.h5")

### autor: Tomasz Sołtysiak